In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import json
%matplotlib inline

In [2]:
dataset = json.load(open('train_01.json'))
dataset

{'inside': [[0.7295200749361322, 0.7286337326606256],
  [0.826505923560258, 0.9403158114766519],
  [0.6023139955320228, 0.4890999051968531],
  [0.7924301535257385, 0.9724603339895345],
  [0.3739062132424882, 0.3807945141554343],
  [0.5213825315815278, 0.7011647516840395],
  [0.5799489554848042, 0.860210492806848],
  [0.4472712889659769, 0.23208680582951224],
  [0.8266928088733442, 0.9708616047497776],
  [0.2096090384196102, 0.07948946448504302],
  [0.33832612728389067, 0.4113630733104253],
  [0.68383264433507, 0.9003136248995234],
  [0.3974079484193441, 0.44643242592599985],
  [0.26193673674112883, 0.20727182114712916]],
 'outside': [[0.846590438950232, 0.12680486596335516],
  [0.11671041192819076, 0.8523469688748586],
  [0.8939436660435063, 0.20441969005621619],
  [0.196515524259539, 0.7563403480339516],
  [0.762462325724514, 0.17675381094088127],
  [0.8646206065391742, 0.4289513713302239],
  [0.7086382515784717, 0.1718070095596751],
  [0.07308462517167347, 0.7068687883532849],
  [0.7

In [10]:
d = []
r = {'inside':1, 'outside':-1}
for k in dataset.keys():
    _df = pd.DataFrame(dataset[k], columns=['A', 'B'])
    _df['class'] = r[k]
    d.append(_df)
df = pd.concat(d, axis=0, ignore_index=True)

In [11]:
df

,A,B,class
0,0.729520,0.728634,1
1,0.826506,0.940316,1
2,0.602314,0.489100,1
3,0.792430,0.972460,1
4,0.373906,0.380795,1
5,0.521383,0.701165,1
6,0.579949,0.860210,1
7,0.447271,0.232087,1
8,0.826693,0.970862,1
9,0.209609,0.079489,1


In [6]:
r[['inside','inside','inside',]]

TypeError: unhashable type: 'list'